In [1]:
import sys
import dawis as d
import glob as glob
import os
import numpy as np
import pyregion as pyr
import random
import gc
import h5py
import dawis
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
from astropy.visualization import *
from scipy.stats import kurtosis
from power_ratio import *
from datetime import datetime
from photutils.segmentation import SourceCatalog, detect_sources
from photutils.background import Background2D, MedianBackground
import make_results_ERO_noray
import tracemalloc
import matplotlib
import cmasher
from astropy.wcs import WCS
from scipy.ndimage import gaussian_filter
from matplotlib.patches import Rectangle
from astropy.stats import sigma_clip, SigmaClip
import subprocess


In [3]:
# Paths, lists & variables
path_data = '/home/aellien/Euclid_ERO/data/Euclid-NISP-Stack-ERO-Abell2390.DR3/Abell2390_NISP_starsubtraction'
path_scripts = '/home/aellien/Euclid_ERO/Euclid_ERO_scripts'
path_wavelets = '/home/aellien/Euclid_ERO/wavelets/out13/'
path_plots = '/home/aellien/Euclid_ERO/plots'
path_analysis = '/home/aellien/Euclid_ERO/analysis/'

In [5]:
# Make same image without masking
nfdl = [ {'infile':'Final_output_JGM_Abellfinal_psf_sub_Hv1_Callummock.fits', 'filter':'H'},
        {'infile':'Final_output_JGM_Abellfinal_psf_sub_Jv1_vC1_v21.fits', 'filter':'J'}, 
        {'infile':'Final_output_JGM_Abellfinal_psf_sub_Hv1_vC1_v24.fits', 'filter':'H'},
        {'infile':'Final_output_JGM_Abellfinal_psf_sub_Yv1_vC1_v21.fits', 'filter':'Y'} ]
for nfd in nfdl:
    
    print(nfd['infile'])
    infile = os.path.join(path_data, nfd['infile'])
    hdu = fits.open(infile)
    oim, header = hdu[0].data, hdu[0].header

    # rot
    rot = infile[:-4]+'rot.fits'
    subprocess.run(['astwarp', '--rotate=295', '-h0', infile, '--output=%s'%rot])

    # crop
    crop = rot[:-4]+'crop.fits'
    subprocess.run(['astcrop', '--mode=wcs', '--center=328.40009,17.699051', '--width=0.75,0.75', '-h1', '--output=%s'%crop, rot])

    # warp
    warp = crop[:-4]+'warp.fits'
    subprocess.run(['astwarp', '--scale=1./4.', '-h1', '--output=%s'%warp, crop])

    # dawis input rdy
    input = warp[:-4]+'input.fits'
    subprocess.run(['astfits', warp, '--copy=1', '--output=%s'%input, '--primaryimghdu'])

print('done')

Final_output_JGM_Abellfinal_psf_sub_Hv1_Callummock.fits
Warp 0.21 started on Mon Jul 29 22:26:03 2024
 Using 20 CPU threads.
 Input: /home/aellien/Euclid_ERO/data/Euclid-NISP-Stack-ERO-Abell2390.DR3/Abell2390_NISP_starsubtraction/Final_output_JGM_Abellfinal_psf_sub_Hv1_Callummock.fits (hdu: 0)
 matrix:
	 0.4226    0.9063    0.0000
	-0.9063    0.4226    0.0000
	 0.0000    0.0000    1.0000
 Output: /home/aellien/Euclid_ERO/data/Euclid-NISP-Stack-ERO-Abell2390.DR3/Abell2390_NISP_starsubtraction/Final_output_JGM_Abellfinal_psf_sub_Hv1_Callummock.rot.fits
Warp finished in:  17.965971 seconds
Crop 0.21 started on Mon Jul 29 22:26:21 2024
  - Read metadata of 1 dataset.                        0.000882 seconds
  ---- ...v1_Callummock.rot.crop.fits created: 1 input.
Crop finished in:  1.138486 seconds
Warp 0.21 started on Mon Jul 29 22:26:22 2024
 Using 20 CPU threads.
 Input: /home/aellien/Euclid_ERO/data/Euclid-NISP-Stack-ERO-Abell2390.DR3/Abell2390_NISP_starsubtraction/Final_output_JGM_Abell